In [17]:
import requests
import json
import pandas as pd
import time
import os

In [2]:
# fetch api key from file

with open('secret/oai_key.txt', 'r') as file:
    api_key = file.read()
print(f'api_key is type {type(api_key).__name__} and is {len(api_key)} characters long')

api_key is type str and is 32 characters long


In [3]:
url = 'https://api.originality.ai/api/v1'

In [22]:
# classify web page and return 1response

def fetch_url_class(url_in): 
    scan_url='https://api.originality.ai/api/v1/scan/url'
    headers={'X-OAI-API-KEY': api_key}
    params={"url": url_in}
    response = requests.post(url=scan_url, headers=headers, params=params)
    return(response)

In [122]:
def extract_score(response):
    r_json = response.json()
    if r_json['success']:
        data = {
        'Success': [r_json['success']],
        'Percent_Human': [round(r_json['score']['original'] * 100, 1)],
        'Percent_AI': [round(r_json['score']['ai'] * 100, 1)]
        }
    else:
        data = {
            'Success': False,
            'Percent_Human': None,
            'Percent_AI': None
        }
    df = pd.DataFrame(data)
    return df

In [120]:
# def extract_score(response):
#     r_json = response.json()
#     data = {
#         'Success': [r_json['success']],
#         'Percent_Human': [round(r_json['score']['original'] * 100, 1)],
#         'Percent_AI': [round(r_json['score']['ai'] * 100, 1)]
#     }
#     df = pd.DataFrame(data)
#     return df

In [123]:
# CLASSIFY ONE WEB PAGE

url_in = "https://www.target.com/c/table-lamps-lighting-home-decor/-/N-56d7t"
response  = fetch_url_class(url_in)
df = extract_score(response)
df

,Success,Percent_Human,Percent_AI
0,True,99.9,0.1


In [ ]:
# Reunite results with original data


In [9]:
# NOW RUN A HUNDRED A MINUTE THROUGH A LOOP
# load 3 csv files
# relaed from csv
df_linked_kw_final = pd.read_csv('data/linked_kw_final.csv', header=0)
# combine csvs
# break 1000 list in batches of 100 and save to csv
# re-add other seo info



In [11]:
for link in df_linked_kw_final:
    df = extract_score(fetch_url_class(url_in))

,kw,rank,link
19985,bike bag,16,https://builtbyswift.com/
19986,bike bag,17,https://frostriver.com/collections/cycling-bik...
19987,bike bag,18,https://www.basil.com/en/bicycle-bags/
19988,bike bag,19,https://topodesigns.com/products/bike-bag
19989,bike bag,20,https://www.duluthpack.com/collections/bike-bags


In [58]:
def make_csv_batches(df, batch_size=5, output_dir='data/batches'):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Get unique values in the 'kw' column
    unique_kw = df['kw'].unique()

    # Break up unique values into batches of batch_size
    kw_batches = [unique_kw[i:i+batch_size] for i in range(0, len(unique_kw), batch_size)]

    # Iterate through kw_batches and create csv files for each batch
    filenames = []
    for i, batch in enumerate(kw_batches):
        # Select rows from df where 'kw' is in the current batch
        batch_df = df[df['kw'].isin(batch)]

        # Write subset_df to csv
        filename = os.path.join(output_dir, f'kw_batch_{i:03d}.csv')
        batch_df.to_csv(filename, index=False)
        filenames.append(filename)

    # Write list of filenames to batch_list.csv
    with open(os.path.join(output_dir, 'batch_list.csv'), 'w') as f:
        f.write('\n'.join(filenames))

    print(f'{len(kw_batches)} csv files created in {output_dir}')

In [59]:
make_csv_batches(df_linked_kw_final)

200 csv files created in data/batches


In [60]:

def read_batch_list(path='data/batches'):
    batch_list_path = os.path.join(path, 'batch_list.csv')
    with open(batch_list_path, 'r') as file:
        batch_filenames = [line.strip() for line in file]
    return batch_filenames

In [63]:
x = read_batch_list()
x[89:100]

['data/batches/kw_batch_089.csv',
 'data/batches/kw_batch_090.csv',
 'data/batches/kw_batch_091.csv',
 'data/batches/kw_batch_092.csv',
 'data/batches/kw_batch_093.csv',
 'data/batches/kw_batch_094.csv',
 'data/batches/kw_batch_095.csv',
 'data/batches/kw_batch_096.csv',
 'data/batches/kw_batch_097.csv',
 'data/batches/kw_batch_098.csv',
 'data/batches/kw_batch_099.csv']

In [18]:
# PROBABLY BROKE BECAUSE OVER 100 RECORDS

# read the df_linked_kw_final DataFrame from file
df_linked_kw_final = pd.read_csv('data/linked_kw_final.csv', header=0)

# apply the extract_score function to each URL in the 'link' column
df_scores = df_linked_kw_final['link'].apply(lambda x: extract_score(classify_url(x)))

# merge the scores DataFrame with the original DataFrame
df_final = pd.concat([df_linked_kw_final, df_scores], axis=1)
